<a href="https://colab.research.google.com/github/alik604/Notebooks/blob/master/Time%20Series/stock_algo_trading_CCI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing the libraries
import math
import datetime

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (15, 9)   # Change the size of plots

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# !pip yfinance
import yfinance as yf
import pandas_datareader.data as web

# https://github.com/atreadw1492/yahoo_fin
# !pip install yahoo_fin
from yahoo_fin.stock_info import * # get_data, tickers_sp500, tickers_nasdaq, tickers_other, get_quote_table 

# !pip install ta
import ta 

In [2]:
start = datetime.datetime(2016,1,1)
# start = datetime.datetime(2020,3,21)
end = datetime.date.today()

# Correct as of 18 may https://www.blackrock.com/ca/investors/en/products/239839/ishares-sptsx-capped-energy-index-etf
symbols = ['CNQ.to', 'SU.to', 'CVE.to', 'IMO.to', 'TOU.to', 'ARX.to', 'WCP.to', 'PSK.to', 'CPG.to', 'PXT.to', 'MEG.to', 'ERF.to', 'VET.to']
weights = [25.51, 24.45, 11.96, 8.41, 7.67, 5.90, 2.80, 2.69, 2.56, 2.55, 2.08, 1.89, 1.50, 0.03]
weights = np.array(weights)/100 
symbol_weights = dict(zip(symbols, weights))

# dataset = dict()
# dataset['XEG.TO'] = yf.download('XEG.TO', start, end)["Adj Close"]

expected_price = 0.0
for symbol in symbols:
#     dataset[symbol] = yf.download(symbol, start, end)["Adj Close"]
    price = get_live_price(symbol)
    expected_price += price * symbol_weights[symbol]
    print(f'{symbol}\t is {price:.2f}.\t Weighted price is {expected_price:.2f}')
    
# stocks = pd.DataFrame(dataset)
print(f'\nExpected price {expected_price:.2f}')

CNQ.to	 is 41.96.	 Weighted price is 10.70
SU.to	 is 28.90.	 Weighted price is 17.77
CVE.to	 is 9.96.	 Weighted price is 18.96
IMO.to	 is 39.48.	 Weighted price is 22.28
TOU.to	 is 29.41.	 Weighted price is 24.54
ARX.to	 is 9.36.	 Weighted price is 25.09
WCP.to	 is 5.86.	 Weighted price is 25.25
PSK.to	 is 13.80.	 Weighted price is 25.62
CPG.to	 is 5.03.	 Weighted price is 25.75
PXT.to	 is 20.49.	 Weighted price is 26.27
MEG.to	 is 7.69.	 Weighted price is 26.43
ERF.to	 is 7.72.	 Weighted price is 26.58
VET.to	 is 9.66.	 Weighted price is 26.73

Expected price 26.73


In [3]:
expected_price

26.725617315816876

In [8]:
get_live_price('XEG.to')

8.345000267028809

In [4]:
# stocks.head(5)
# stocks.tail(5)

In [5]:
# stocks.shape[0]
# stocks=stocks.dropna()
# stocks.shape[0]
# stocks.plot(grid=True, figsize=(18, 6))

In [6]:
# def normIt(myDf):
#    return (myDf - myDf.mean()) / (myDf.max() - myDf.min())

# stocks_norm = normIt(stocks)
# stocks_norm.shape[0]
# stocks_norm= stocks_norm.dropna()
# stocks_norm.shape[0]
# stocks_norm.plot(grid=True, figsize=(18, 6))